<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from ast import literal_eval
import pandas as pd
import math
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1idXUpsWYnPP3ou6W-3h6moWSvgWDBZs7'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('PopularMovies.csv') 
movies_data = pd.read_csv('PopularMovies.csv',engine='python',index_col=0)
movies_data.head()

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,-1.082731e+09,0.634881,2015,Wednesday,"['assistant', 'aftercreditsstinger', 'duringcr...",Family|Animation|Adventure|Comedy,United States of America,Universal Pictures|Illumination Entertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda
1,177572,Big Hero 6,165000000,-4.871054e+08,0.598368,2014,Friday,"['brother brother relationship', 'hero', 'tale...",Adventure|Family|Animation|Action|Comedy,United States of America,Walt Disney Pictures|Walt Disney Animation Stu...,English,102.0,The special bond that develops between plus-si...,Released,Scott Adsit|Ryan Potter|Daniel Henney|T.J. Mil...,Chris Williams
2,155,The Dark Knight,185000000,-8.195584e+08,0.565743,2008,Wednesday,"['dc comics', 'crime fighter', 'secret identit...",Drama|Action|Crime|Thriller,United Kingdom|United States of America,DC Comics|Legendary Pictures|Warner Bros.|DC E...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan
3,680,Pulp Fiction,8000000,-2.059288e+08,0.558571,1994,Saturday,"['transporter', 'brothel', 'drug dealer', 'box...",Thriller|Crime,United States of America,Miramax Films|A Band Apart|Jersey Films,English,154.0,"A burger-loving hit man, his philosophical par...",Released,John Travolta|Samuel L. Jackson|Uma Thurman|Br...,Quentin Tarantino
4,278,The Shawshank Redemption,25000000,-3.341469e+06,0.554077,1994,Friday,"['prison', 'corruption', 'police brutality', '...",Drama|Crime,United States of America,Castle Rock Entertainment|Warner Bros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,Tim Robbins|Morgan Freeman|Bob Gunton|Clancy B...,Frank Darabont


In [4]:
movies_data.shape

(9081, 17)

In [5]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9046 non-null   object 
 9   production_countries  8787 non-null   object 
 10  production_companies  8417 non-null   object 
 11  original_language     9079 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9069 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

In [6]:
movies_data=movies_data.replace(np.nan,'Unknown',regex=True)
movies_data['release_year']=movies_data['release_year'].replace('NaT','1874',regex=True)

In [7]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9081 non-null   object 
 9   production_countries  9081 non-null   object 
 10  production_companies  9081 non-null   object 
 11  original_language     9081 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9081 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

In [8]:
movies_data['production_companies']=movies_data['production_companies'].str.replace(" ","")
movies_data['production_companies']=movies_data['production_companies'].str.replace("|","  ")
movies_data['production_countries']=movies_data['production_countries'].str.replace(" ","")
movies_data['production_countries']=movies_data['production_countries'].str.replace("|","  ")
movies_data['cast']=movies_data['cast'].str.replace(" ","")
movies_data['cast']=movies_data['cast'].str.replace("|","  ")
movies_data['genres']=movies_data['genres'].str.replace("|"," ")
movies_data['director']=movies_data['director'].str.replace(" ","")
movies_data['keywords']=movies_data['keywords'].apply(lambda x:x[1:-1])
movies_data['keywords']=movies_data['keywords'].str.replace(',',"  ")
movies_data['keywords']=movies_data['keywords'].str.replace("'","")
movies_data.head(10)

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,-1.082731e+09,0.634881,2015,Wednesday,assistant aftercreditsstinger duringcredit...,Family Animation Adventure Comedy,UnitedStatesofAmerica,UniversalPictures IlluminationEntertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,SandraBullock JonHamm MichaelKeaton Allison...,KyleBalda
1,177572,Big Hero 6,165000000,-4.871054e+08,0.598368,2014,Friday,brother brother relationship hero talent ...,Adventure Family Animation Action Comedy,UnitedStatesofAmerica,WaltDisneyPictures WaltDisneyAnimationStudios,English,102.0,The special bond that develops between plus-si...,Released,ScottAdsit RyanPotter DanielHenney T.J.Mill...,ChrisWilliams
2,155,The Dark Knight,185000000,-8.195584e+08,0.565743,2008,Wednesday,dc comics crime fighter secret identity ...,Drama Action Crime Thriller,UnitedKingdom UnitedStatesofAmerica,DCComics LegendaryPictures WarnerBros. DCEn...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,ChristianBale MichaelCaine HeathLedger Aaro...,ChristopherNolan
3,680,Pulp Fiction,8000000,-2.059288e+08,0.558571,1994,Saturday,transporter brothel drug dealer boxer ...,Thriller Crime,UnitedStatesofAmerica,MiramaxFilms ABandApart JerseyFilms,English,154.0,"A burger-loving hit man, his philosophical par...",Released,JohnTravolta SamuelL.Jackson UmaThurman Bru...,QuentinTarantino
4,278,The Shawshank Redemption,25000000,-3.341469e+06,0.554077,1994,Friday,prison corruption police brutality priso...,Drama Crime,UnitedStatesofAmerica,CastleRockEntertainment WarnerBros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,TimRobbins MorganFreeman BobGunton ClancyBr...,FrankDarabont
5,210577,Gone Girl,61000000,-3.083304e+08,0.523202,2014,Wednesday,based on novel marriage crisis disappearan...,Mystery Thriller Drama,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,145.0,With his wife's disappearance having become th...,Released,BenAffleck RosamundPike CarrieCoon NeilPatr...,DavidFincher
6,293660,Deadpool,58000000,-7.251130e+08,0.518324,2016,Tuesday,anti hero mercenary marvel comic superhe...,Action Adventure Comedy,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation MarvelEnte...,English,108.0,Deadpool tells the origin story of former Spec...,Released,RyanReynolds MorenaBaccarin EdSkrein T.J.Mi...,TimMiller
7,157336,Interstellar,165000000,-5.101200e+08,0.514384,2014,Wednesday,saving the world artificial intelligence f...,Adventure Drama Science Fiction,Canada UnitedStatesofAmerica UnitedKingdom,ParamountPictures LegendaryPictures WarnerBr...,English,169.0,Interstellar chronicles the adventures of a gr...,Released,MatthewMcConaughey JessicaChastain AnneHatha...,ChristopherNolan
8,550,Fight Club,63000000,-3.785375e+07,0.495441,1999,Friday,support group dual identity nihilism rag...,Drama,Germany UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,139.0,A ticking-time-bomb insomniac and a slippery s...,Released,EdwardNorton BradPitt MeatLoaf JaredLeto H...,DavidFincher
9,118340,Guardians of the Galaxy,170000000,-6.033286e+08,0.484757,2014,Wednesday,marvel comic spaceship space outer space...,Action Science Fiction Adventure,UnitedKingdom UnitedStatesofAmerica,MarvelStudios MovingPictureCompany(MPC) Bull...,English,121.0,"Light years from Earth, 26 years after being a...",Released,ChrisPratt ZoeSaldana DaveBautista VinDiese...,JamesGunn


In [9]:
movies_data['keywords']=movies_data['keywords']+" " +movies_data['cast']+" "+movies_data['production_companies']+" "+movies_data['production_countries']+" "+movies_data['genres']+" "+movies_data['original_language']+" "+movies_data['release_day']+" "+movies_data['content']+" "+movies_data['director']
movies_data['keywords']=movies_data['keywords'].replace({"[^A-Za-z0-9]+": " "},regex=True)
movies_data['keywords'][0]

'assistant aftercreditsstinger duringcreditsstinger evil mastermind minions 3d SandraBullock JonHamm MichaelKeaton AllisonJanney SteveCoogan JenniferSaunders GeoffreyRush SteveCarell PierreCoffin KatyMixon MichaelBeattie HiroyukiSanada DaveRosenbaum AlexDowding PaulThornley KyleBalda AvaAcres UniversalPictures IlluminationEntertainment UnitedStatesofAmerica Family Animation Adventure Comedy English Wednesday Minions Stuart Kevin and Bob are recruited by Scarlet Overkill a super villain who alongside her inventor husband Herb hatches a plot to take over the world Before Gru they had a history of bad bosses KyleBalda'

In [10]:
vectorizer=TfidfVectorizer(stop_words='english')
X=vectorizer.fit_transform(movies_data['keywords'])
tfidf_df=pd.DataFrame.sparse.from_spmatrix(X)

In [12]:
movies_df=movies_data[['tmdbId','budget','profit','score','release_year','runtime']]
movies_df=movies_df.join(tfidf_df)
movies_df.set_index('tmdbId',inplace=True)
movies_df.shape

(9081, 124699)